# Lab Code Generation 

- It is recommended to **go through this file with a partner**. 
- Ensure to **ask** if anything is not clear - first your partner, then a lab helper.
- You want to first go through the accompanying code of the lectures.

In [ ]:
#require "jupyter.notebook" ;;

open Jupyter_notebook;;

Process.sh "ocamllex simp.mll";;
Process.sh "ocamlc -c simp.ml";;

#use "psimp.ml";;

(* Definition of MIPS *)

(* We represent registers as numbers. Registers are represented by 0 to 31. *)
type register = int

(* Value returned by a subroutine *)
let v0 : register = 2 
let v1 : register = 3 

(* Arguments to subroutine *)
let a0 : register = 4 
let a1 : register = 5
let a2 : register = 6
let a3 : register = 7

(* Temporary registers *)
let t0 : register = 8
let t1 : register = 9
let t2 : register = 10
let t3 : register = 11
let t4 : register = 12
let t5 : register = 13
let t6 : register = 14
let t7 : register = 15

(* Saved registers *)
let s0 : register = 16
let s1 : register = 17 
let s2 : register = 18 
let s3 : register = 19 
let s4 : register = 20 
let s5 : register = 21
let s6 : register = 22 
let s7 : register = 23 

(* Temporary registers $t8 and $t9 will be used for intermediate results. *)
let t8 : register = 24 (* $t8 *)
let t9 : register = 25 (* $t9 *)

let (sp : register) = 29 (* stack pointer *)
let (fp : register) = 30 (* frame pointer *)
let (ra : register) = 31 (* return address *)

(* We represent instructions as an abstract data type. *)

type label = string

type instruction =  Add of register * register * register (* add $1, $2, $3 - $1 = $2 + $3 *)
                   | Sub of register * register * register (* sub $1, $2, $3; $1 = $2 - $3 *)
                   | Addi of register * register * int (* addi $1, $2, 100 - $1 = $2 + 100, immediate means a constant number  *)
                   | Addiu of register * register * int (* addi $1, $2, 100 - $1 = $2 + 100, values treated as unsigned, immediate means a constant number  *)
                   | Mul of register * register * register (* mul $1, $2, $3 - $1 = $2 * $3, without overflow, result is only 32 bits *)
                   | Div of register * register (* div $2, $3 - $hi,$low=$2/$3, Remainder stored in special register hi, Quotient stored in special register lo   *)
                   | And of register * register * register (* and $1, $2, $3 - $1 = $2 & $3, bitwise AND *)
                   | Or of register * register * register (* or $1, $2, $3 - $1 = $2 | 100, bitwise OR *)
                   | Andi of register * register * int (* andi $1, $2, 100 - $1 = $2 & 100, bitwise AND with immediate value  *)
                   | Ori of register * register * int (* ori $1, $2, 100 - $1 = $2 | 100, bitwise OR with immediate value  *)
                   | Lw of register * int * register (* lw $1, 100 ($2) - load word, $1 = Memory[$2 + 100], copy from memory to register *)
                   | Sw of register * int * register (* sw $1, 100 ($2) - store word, Memory[$2 + 100] = $1, copy from register to memory *)
                   | La of register * label (* $1 = Address of label *) 
                   | Li of register * int (* li $1, 100 - Loads immediate value into register *)
                   | Move of register * register (* move $1,$2 - $1 = $2, Copy from register to register *)
                   | Mfhi of register (* mfhi $2, $2 = hi, copy from special register hi to general register *)
                   | Mflo of register (* mflo $2, $2 = lo, copy from special register lo to general register *)
                   | Label of label 
                   | Beq of register * register * string (* beq $1, $2, l - if ($1 == $2) go to label l *)
                   | Bne of register * register * string (* bne $1, $2, l - if ($1 != $2) go to label l *)
                   | Bgt of register * register * string (* bgt $1, $2, l - if ($1 > $2) go to label l *)
                   | Blt of register * register * string (* blt $1, $2, l - if ($1 < $2) go to label l *)
                   | Bge of register * register * string (* bge $1, $2, l - if ($1 >= $2) go to label l *)
                   | Ble of register * register * string (* ble $1, $2, l - if ($1 <= $2) go to label l *)                  
                   | J of label (* j l, go to label l, jumps to target address *)
                   | Jr of register (* jump register, jr $1, go to address stored in $1 *)
                   | Jal of label (* jump and link, e.g. jal l - $ra=PC+4; go to label l - used when making procedure call. This saves the return address in $ra.  *)
                   | SysCall 
                   | Verbatim of string (* Produce the given string verbatim in the assembly output *)
                   
type code = instruction list

let print_register (r : register) = 
    match r with 
    | 2 | 3 -> "$v" ^ (string_of_int (r - v0)) 
    | 4 |5 |6 | 7 -> "$a" ^ string_of_int (r - a0)
    | 8|9|10|11|12|13|14|15 -> "$t" ^ string_of_int (r - t0) 
    | 16|17|18|19|20|21|22|23 -> "$s" ^ string_of_int (r - s0)
    | 24 -> "$t8"
    | 25 -> "$t9"
    | 29 -> "$sp"
    | 30 -> "$fp"
    | 31 -> "$ra"
    | _ -> "$" ^ string_of_int r

let print_instruction (i : instruction) = match i with 
    | Add (r1, r2, r3) -> "add " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ print_register r3
    | Sub (r1, r2, r3) -> "sub " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ print_register r3
    | Addi (r1, r2, i) -> "addi " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ string_of_int i
    | Addiu (r1, r2, i) -> "addiu " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ string_of_int i
    | Mul (r1, r2, r3) -> "mul " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ print_register r3
    | Div (r1, r2) -> "div " ^ print_register r1 ^ ", " ^ print_register r2
    | Beq (r1, r2, l) ->  "beq " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ l
    | Bne (r1, r2, l) ->  "bne " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ l
    | Bgt (r1, r2, l) ->  "bgt " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ l
    | Blt (r1, r2, l) ->  "blt " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ l
    | Bge (r1, r2, l) ->  "bge " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ l
    | Ble (r1, r2, l) ->  "ble " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ l
    | Li (r, i) -> "li " ^ print_register r ^ ", " ^ string_of_int i
    | Lw (r1, o, r2) -> "lw " ^ print_register r1 ^ ", " ^ string_of_int o ^ "(" ^ print_register r2 ^ ")" 
    | La (r, l) -> "la " ^ print_register r ^ ", " ^  l
    | Sw (r1, o, r2) -> "sw " ^ print_register r1 ^ ", " ^ string_of_int o ^ "(" ^ print_register r2 ^ ")" 
    | Move (r1, r2) -> "move " ^ print_register r1 ^ ", " ^ print_register r2
    | Mfhi r -> "mfhi "^ print_register r
    | Mflo r -> "mflo "^ print_register r
    | And (r1, r2, r3) -> "and " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ print_register r3
    | Andi (r1, r2, r3) -> "andi " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ print_register r3
    | Or (r1, r2, r3) -> "or " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ print_register r3
    | Ori (r1, r2, r3) -> "ori " ^ print_register r1 ^ ", " ^ print_register r2 ^ ", " ^ print_register r3
    | SysCall -> "syscall"
    | Label l -> l ^ ":"
    | J label -> "j " ^ label 
    | Jr r -> "jr " ^ print_register r
    | Jal label -> "jal " ^ label
    | Verbatim s -> s
   
let rec print_code (c : code) : unit = match c with 
    | [] -> ()
    | c :: cs -> (print_endline (print_instruction c); print_code cs)
    
exception EEnv of string

let maxreg = 23

module Env = Map.Make (String)

(*  Function that finds the largest register in the environment. *)
let find_max_register (env : int Env.t) = 
    Env.fold (fun _ a b -> max a b) env (s0 - 1) 


(* Declaring a variable: 
  - When trying to declare a variable, and there are too many registers already 
    reserved, throw an exception. 
  - Else: Assign to variable x the largest register number + 1.
*)
let declare_var (env: int Env.t) (x:string) : int Env.t = 
    if (find_max_register env) >= maxreg
     then raise (EEnv "Too many variables")
     else Env.add x (1 + find_max_register env) env
     
exception E of string

(* Pushing the content of register r to the stack *)
let push (r : register) : code = [Addiu (sp, sp, -4);
                                  Sw (r, 0, sp)]

(* Popping the stack into register r *)
let pop (r : register) : code = [Lw (r, 0, sp);
                                 Addiu (sp, sp, 4)]
                                 
                                 
let counter : int ref = {contents = 0}

let next_val = 
    fun () ->
      counter := (!counter) + 1;
      !counter;;                                 

## 1. SIMP Compilation 

1. You are given the following program ``p1``: 
```
VAR A; VAR B; VAR C; 
A := 3; B := 4; 
BEGIN 
	VAR B; VAR C;
	B := 3; 
	C := 4
END 
C := 5
```

Which of the following are valid compilations of the program? 

``` 
1a) li $s0, 3
li $s1, 4
li $s1, 3
li $s2, 4
li $s2, 5

1b) li $s0, 3
li $s1, 4
li $s3, 3
li $s4, 4
li $s4, 5

1c) li $s0, 3
li $s1, 4
li $s3, 3
li $s4, 4
li $s2, 5
```

2. You are given the following program 
```
IF 3 > 2 THEN PRINT 3 
```

What are valid compilations of the program? 

```
2a) li $t8, 3
addiu $sp, $sp, -4
sw $t8, 0($sp)
li $t9, 2
lw $t8, 0($sp)
addiu $sp, $sp, 4
ble $t8, $t9, IFEND1
li $v0, 1
li $a0, 3
syscall
IFEND1:
li $v0, 10
syscall

2b) li $t8, 3
addiu $sp, $sp, -4
sw $t8, 0($sp)
li $t9, 2
lw $t8, 0($sp)
addiu $sp, $sp, 4
bgt $t8, $t9, IFEND1
li $v0, 1
li $a0, 3
syscall
IFEND1:
li $v0, 10
syscall

2c) li $v0, 1
li $a0, 3
syscall
IFEND1:
li $v0, 10
syscall
```


In [ ]:
from jupyterquiz import display_quiz

question_path="./"

display_quiz(question_path+"questions4.json")

## 2. Compiling New Constructs

1. **Compiling If-Then-Else** 

a.) Write a program that saves a maximum of two variables ``a`` (in ``$s0``) and b (``$s1``) into the register of ``max`` (``$s2``):

```
VAR a;
VAR b;
VAR max;
…
IF (a > b)
THEN max := a 
ELSE max := b; 
...
```

b.) Write a list of the steps needed to compile ``IF (e1 comp e2) THEN c1 ELSE c2``. 
  E.g., "1. Code for command ``c1``."

c.) Complement the function ``compile_ITE`` in the below code such that ``compile_ITE e c1 c2`` puts out the code to compile ``IF e THEN c1 ELSE c2``. 

d.) Test that your compilation works, by 1.) compiling the following program and 2.) testing it yields the right result in the emulator (https://cpulator.01xz.net/?sys=mipsr5b-spim).

  ```
  VAR a; VAR b; 
  INPUT a; 
  INPUT b; 
  IF a>b
  THEN PRINT a ELSE PRINT b
  ```



In [ ]:
let compile_op (o : op) r1 r2 r = match o with 
    | Plus -> [Add (r1, r2, r)]
    | Minus -> [Sub (r1, r2, r)]
    | Mult -> [Mul (r1, r2, r)] 
    | Div -> [Div (r1, r2); Mflo r] 

let rec compile_exp env (r : register) (e : exp) : code = match e with 
    | Id s -> [Move (r, Env.find s env)]
    | Numb n -> [Li (r, n)]
    | Neg e -> compile_exp env r (Op (Numb 0, Minus, e))
    | Op (e1, o, e2) -> compile_exp env t8 e1
                        @ push t8
                        @ compile_exp env t9 e2
                        @ pop t8
                        @ compile_op o r t8 t9;;


let compile_branch o r1 r2 label = match o with 
    | Lt -> Bge (r1, r2, label)
    | Lte -> Bgt (r1, r2, label)
    | Gte -> Blt (r1, r2, label)
    | Gt -> Ble (r1, r2, label)
    | Eq -> Bne (r1, r2, label)
    | Neq -> Beq (r1, r2, label)
    
let compile_cond env o e1 e2 label =   
    compile_exp env t8 e1
    @ push t8
    @ compile_exp env t9 e2
    @ pop t8
    @ [compile_branch o t8 t9 label]
    

let rec compile_dcls env dcls = match dcls with 
    | [] -> env 
    | dcl :: dcls' -> let env' = declare_var env dcl 
                      in compile_dcls env' dcls'

let rec compile_cmd env (c : cmd) : code = match c with 
    | If (e, c) -> (let label = "IFEND" ^ string_of_int (next_val ()) in
                    match e with 
                    | Cop (e1, o, e2) -> compile_cond env o e1 e2 label
                                        @ compile_cmd env c
                                        @ [Label label])
    | Ite (e, c1, c2) -> compile_ITE env e c1 c2
    | While (e, c) -> ( let counter = string_of_int (next_val ()) in
                        let label_loop = "WLOOP" ^ counter in
                        let label_end = "WEND" ^ counter in
                match e with 
                | Cop (e1, o, e2) -> [Label label_loop]
                                    @ compile_cond env o e1 e2 label_end
                                    @ compile_cmd env c
                                    @ [J label_loop]
                                    @ [Label label_end])
    | Asgn (s, e) -> compile_exp env (Env.find s env) e
    | Begin (Program (dcls, cmds)) -> let env' = compile_dcls env dcls in
                                      compile_cmds env' cmds 
    | Print e -> [Li (v0, 1)] 
                 @ compile_exp env a0 e
                 @ [SysCall]
    | Input s -> let s_register = Env.find s env in 
                 [Li (v0, 4); La (a0, "sinp"); SysCall; Li (v0, 5); SysCall; Move (s_register, v0)]

and compile_cmds env (cs : cmd list) : code = match cs with 
    | [] -> [] 
    | c :: cs -> compile_cmd env c @ compile_cmds env cs
    
and compile_ITE env e c1 c2 = [] (* TODO: IMPLEMENT. *)


let compile_program (p : program) = match p with 
    | Program (dcls, cmds) -> let env = compile_dcls Env.empty dcls in     
        [Verbatim ".set noreorder"; Verbatim ".data"; Label "sinp"; Verbatim ".asciiz \"INPUT>\"  "; Verbatim ".text"; Label "_start"]
        @ compile_cmds env cmds
        @ [Li (v0, 10); SysCall]

In [ ]:
let test = "  VAR a; VAR b; 
  INPUT a; 
  INPUT b; 
  IF a>b
  THEN PRINT a ELSE PRINT b"
  
let mips =  (print_code (compile_program (parse_string test)));;

2. **Compiling ``DoWhile`` Loop**

In Lab 1, you have encountered and extended commands with the concept of a ``DoWhile`` loop.

Recall the following explanation from Wikipedia: 
*The do while construct consists of a process symbol and a condition. First the code within the block is executed. Then the condition is evaluated. If the condition is true the code within the block is executed again. This repeats until the condition becomes false.*

Extend the above definition of commands by the DoWhile loop: 

```
type cmd = ...
         | DoWhile of cmd * condexp
```

... and extend the above code generator. 

Write your own program to test the ``DoWhile`` loop. (You will have to write a program in the abstract syntax.)

In [ ]:
type op = Plus | Minus | Mult | Div 

type exp = Id of string | Numb of int | Op of exp * op * exp | Neg of exp

type cond = Eq | Neq | Lte | Lt | Gte | Gt 
type condexp = Cop of exp * cond * exp
                                          
type cmd = Asgn of string * exp 
         | Ite of condexp * cmd * cmd | If of condexp * cmd 
         | While of condexp * cmd
         | Begin of program 
         | Input of string
         | Print of exp 
         
and program = Program of string list * cmd list

let compile_op (o : op) r1 r2 r = match o with 
    | Plus -> [Add (r1, r2, r)]
    | Minus -> [Sub (r1, r2, r)]
    | Mult -> [Mul (r1, r2, r)] 
    | Div -> [Div (r1, r2); Mflo r] 

let rec compile_exp env (r : register) (e : exp) : code = match e with 
    | Id s -> [Move (r, Env.find s env)]
    | Numb n -> [Li (r, n)]
    | Neg e -> compile_exp env r (Op (Numb 0, Minus, e))
    | Op (e1, o, e2) -> compile_exp env t8 e1
                        @ push t8
                        @ compile_exp env t9 e2
                        @ pop t8
                        @ compile_op o r t8 t9;;


let compile_branch o r1 r2 label = match o with 
    | Lt -> Bge (r1, r2, label)
    | Lte -> Bgt (r1, r2, label)
    | Gte -> Blt (r1, r2, label)
    | Gt -> Ble (r1, r2, label)
    | Eq -> Bne (r1, r2, label)
    | Neq -> Beq (r1, r2, label)
    
let compile_cond env o e1 e2 label =   
    compile_exp env t8 e1
    @ push t8
    @ compile_exp env t9 e2
    @ pop t8
    @ [compile_branch o t8 t9 label]
    

let rec compile_dcls env dcls = match dcls with 
    | [] -> env 
    | dcl :: dcls' -> let env' = declare_var env dcl 
                      in compile_dcls env' dcls'

let rec compile_cmd env (c : cmd) : code = match c with 
    | If (e, c) -> (let label = "IFEND" ^ string_of_int (next_val ()) in
                    match e with 
                    | Cop (e1, o, e2) -> compile_cond env o e1 e2 label
                                        @ compile_cmd env c
                                        @ [Label label])
    | Ite (e, c1, c2) -> compile_ITE env e c1 c2
    | While (e, c) -> ( let counter = string_of_int (next_val ()) in
                        let label_loop = "WLOOP" ^ counter in
                        let label_end = "WEND" ^ counter in
                match e with 
                | Cop (e1, o, e2) -> [Label label_loop]
                                    @ compile_cond env o e1 e2 label_end
                                    @ compile_cmd env c
                                    @ [J label_loop]
                                    @ [Label label_end])
    | Asgn (s, e) -> compile_exp env (Env.find s env) e
    | Begin (Program (dcls, cmds)) -> let env' = compile_dcls env dcls in
                                      compile_cmds env' cmds 
    | Print e -> [Li (v0, 1)] 
                 @ compile_exp env a0 e
                 @ [SysCall]
    | Input s -> let s_register = Env.find s env in 
                 [Li (v0, 4); La (a0, "sinp"); SysCall; Li (v0, 5); SysCall; Move (s_register, v0)]

and compile_cmds env (cs : cmd list) : code = match cs with 
    | [] -> [] 
    | c :: cs -> compile_cmd env c @ compile_cmds env cs
    
and compile_ITE env e c1 c2 = (let counter = string_of_int (next_val ()) in
                            let label_end = "IFEND" ^ counter in
                            let label_false = "IFFALSE" ^ counter in
                match e with 
                | Cop (e1, o, e2) -> compile_cond env o e1 e2 label_false
                                    @ compile_cmd env c1
                                    @ [J label_end]
                                    @ [Label label_false]
                                    @ compile_cmd env c2
                                    @ [Label label_end])


let compile_program (p : program) = match p with 
    | Program (dcls, cmds) -> let env = compile_dcls Env.empty dcls in     
        [Verbatim ".set noreorder"; Verbatim ".data"; Label "sinp"; Verbatim ".asciiz \"INPUT>\"  "; Verbatim ".text"; Label "_start"]
        @ compile_cmds env cmds
        @ [Li (v0, 10); SysCall]

3. **Compile a switch statement**

See the following description of Wikipedia: 

*In [computer programming languages](https://en.wikipedia.org/wiki/Computer_programming_language), a **switch statement** is a type of selection control mechanism used to allow the value of a [variable](https://en.wikipedia.org/wiki/Variable_(programming)) or expression to change the [control flow](https://en.wikipedia.org/wiki/Control_flow) of program execution.*

E.g., the following is an example program yields the number of days for the months: 

```
switch (month) {
  case 1:  days := 31;            
  case 2:  days := 28;          
  case 3:  days := 31;
  case 4:  days := 30;            
  case 5:  days := 31;          
  case 6:  days := 30;
  case 7:  days := 31;  
  case 8:  days := 31;  
  case 9:  days := 30;            
  case 10:  days := 31;          
  case 11:  days := 30;
  case 12:  days := 31;  
	default: days := 0;
}
```

We represent a switch statement by the following statement: 

```
type cmd = ... 
| Switch of exp (* the expression to be matched on *) 
* (int * cmd) list (* case and command in case this case fits *) 
* cmd (* a command for the default case *)
```

Extend the compiler from 2. with code for a switch statement and test it. 

In [ ]:
print_code (compile_program (Program (["month"; "days"], 
[Input "month"; 
Switch (Id "month", [
(1, Asgn ("days", Numb 31));
(2, Asgn ("days", Numb 28));
(3, Asgn ("days", Numb 31));
(4, Asgn ("days", Numb 30));
(5, Asgn ("days", Numb 31));
(6, Asgn ("days", Numb 30));
(7, Asgn ("days", Numb 31));
(8, Asgn ("days", Numb 31));
(9, Asgn ("days", Numb 30));
(10, Asgn ("days", Numb 31));
(11, Asgn ("days", Numb 30));
(12, Asgn ("days", Numb 31))
],
Asgn ("days", Numb 0));
Print (Id "days")
])))

## 3. Optimization 

1. **Peephole Optimization** 

Look at the following code: 
```
VAR x; 
INPUT x; 
x := 2 * x; 
PRINT x
```

a.) Run the code generator with peephole optimization.
Why is the code not optimal?
Can you remove some of the instructions? 

*Hint:* Recall the optimizations on the original code generator. Are there superfluous ``move`` instructions? You may restrict yourself to the ``Mul`` case.

Extend peephole optimization with the resepctive case.

In [ ]:
(* Peephole Optimization *)

let rec peephole_step (cs : code) : code = match cs with 
| Addiu (29, 29, -4) :: Sw (24, 0, 29) :: 
Li (r, n) :: Lw (24, 0, 29) :: Addiu (29, 29, 4) :: cs' -> if (r = 24) 
                    then Addiu (29, 29, -4) :: Sw (24, 0, 29) :: Li (r, n) :: Lw (24, 0, 29) :: Addiu (29, 29, 4) :: peephole_step cs'
                    else Li (r, n) :: peephole_step cs' 
| Addiu (29, 29, -4) :: Sw (24, 0, 29) :: 
Move (r1, r2) :: Lw (24, 0, 29) :: Addiu (29, 29, 4) :: cs' -> 
                    if (r1 = 24) 
                    then Addiu (29, 29, -4) :: Sw (24, 0, 29) :: Move (r1, r2) :: Lw (24, 0, 29) :: Addiu (29, 29, 4) :: peephole_step cs'
                    else Move (r1, r2) :: peephole_step cs' 
| c :: cs' -> c :: peephole_step cs'
| [] -> []

let rec peephole cs = let cs' = peephole_step cs in 
    if (cs = cs') then cs else peephole cs'

In [ ]:
(* We use directly the abstract syntax tree as we previously in this file changed the definitions. *)

let example_peephole  = Program (["x"],
[Input "x"; Asgn ("x", Op (Numb 2, Mult, Id "x")); Print (Id "x")])

let mips_peephole =  print_code (peephole (compile_program example_peephole))

b.)  You know that multiplication instructions take much longer than addition instructions. 
Extend the above definition of peephole optimization to exchange multiplication with the constant ``2`` to an expression with addition.

Compare the output.

2. **Constant Folding** 

Look at the following program: 
```
VAR x; VAR y;
IF (2 < 3) THEN x := 1 ELSE x := 2
```

In [ ]:
type op = Plus | Minus | Mult | Div 

type exp = Id of string | Numb of int | Op of exp * op * exp | Neg of exp

type cond = Eq | Neq | Lte | Lt | Gte | Gt 
type condexp = Cop of exp * cond * exp 
                                          
type cmd = Asgn of string * exp 
         | Ite of condexp * cmd * cmd | If of condexp * cmd 
         | While of condexp * cmd
         | Begin of program 
         | Input of string
         | Print of exp 
         
and program = Program of string list * cmd list

In [ ]:
let example = Program (["x"; "y"], 
[Ite (Cop (Numb 2, Lt, Numb 3), Asgn ("x", Numb 1), Asgn ("y", Numb 2))])

a.) Why is the program not optimal? How could you optimize it? 

b.) Explain why this optimization is a kind of constant-folding. 

c.) To constant-fold the conditional statements, we will need a way to represent conditions that are already true. Extend the above definition of condition expressions with a boolean constant ``B of bool`` that carries with it the information of a bool. 
Further extend commands with an ``Idle`` command that does nothing (this will be needed to constant-fold the ``If`` command).

d.) Define constant-folding and test your program.

In [ ]:
(* One-step constant folding *)
let rec cf_exp (e : exp)  = match e with 
    | Op (Numb n1, o, Numb n2) -> (match o with 
                                   | Plus -> Numb (n1 + n2)
                                   | Minus -> Numb (n1 - n2)
                                   | Mult -> Numb (n1 * n2)
                                   | Div -> Numb (n1 / n2)
                                    )
    | Op (e1, o, e2) -> Op (cf_exp e1, o, cf_exp e2)
    | Neg (Numb n) -> Numb (-n)
    | Neg e -> Neg (cf_exp e)
    | _ -> e

let cf_condexp (e : condexp) : condexp = match e with 
    | Cop (Numb n1, o, Numb n2) -> (* TODO *)
    | Cop (e1, o, e2) -> Cop (cf_exp e1, o, cf_exp e2)
    | _ -> e

let rec cf_cmd (c : cmd) : cmd = match c with 
    | Asgn (x, e) -> Asgn (x, cf_exp e)
    (* TODO: You will want cases for If/Ite/While here *) 
    | Ite (e, c1, c2) -> Ite (cf_condexp e, cf_cmd c1, cf_cmd c2)
    | If (e, c) -> If (cf_condexp e, cf_cmd c)
    | While (e, c) -> While (cf_condexp e, cf_cmd c)
    | Begin p -> Begin (cf_program p)
    | Print e -> Print (cf_exp e)
    | _ -> c

and cf_program (p : program) : program = match p with 
    | Program (dcls, cs) -> Program (dcls, List.map cf_cmd cs)


(* Multi-step constant-folding - we apply the step function until the program no longer changes. *)
let rec cf p = 
    let p' = cf_program p in 
    if (p = p') then p else cf p'

In [ ]:
cf example

## Challenge

In the lecture, you've seen the definition of constant propagation. 
Extend constant propagation to 1.) ``while``-loops and 2.) ``block``s.
Extend the code of constant propagation (see the notebook from the lecture). 
The following considerations might be helpful: 
- For while loops: When is the condition executed? For which variables is it possible to do constant propagation in the condition? For which variables is it possible to do constant propagation in the body of the loop?
- For blocks: What happens with variables which have been declared before? What happens with variables 
- Look at the following code:
```
VAR x; VAR y; 
INPUT x;
INPUT y;
IF x = 0 THEN 
   y := x 
ELSE 
   y := 0; 
PRINT y
```

Extend constant propagation such that the optimized code changes the last statement to 
```
PRINT 0
```